<a href="https://colab.research.google.com/github/torontodeveloper/homecredit_loan_classification/blob/develop/src/homecredit_default_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [5]:
! pip install kaggle

In [6]:
!mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle

In [8]:
!chmod 777 ~/.kaggle/kaggle.json

In [9]:
! kaggle competitions download home-credit-credit-risk-model-stability

 99% 3.12G/3.14G [00:30<00:00, 172MB/s]
100% 3.14G/3.14G [00:30<00:00, 111MB/s]


In [ ]:
!unzip home-credit-credit-risk-model-stability.zip

In [16]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [19]:
train_basetable = pd.read_csv("csv_files/train/train_base.csv")

train_basetable.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [20]:
test_basetable = pd.read_csv('csv_files/test/test_base.csv')
test_basetable.head()

,case_id,date_decision,MONTH,WEEK_NUM
0,57543,2021-05-14,202201,100
1,57549,2022-01-17,202201,100
2,57551,2020-11-27,202201,100
3,57552,2020-11-27,202201,100
4,57569,2021-12-20,202201,100
